# Regression with Multiple Features — Level 2: scikit-learn

In this notebook, you will re-implement multiple feature regression using **scikit-learn** and compare with your Level 1 (from-scratch) implementation.

### Key scikit-learn Classes

| Class/Function | Purpose |
|---------------|--------|
| `LinearRegression` | Fits a linear model (handles multiple features natively) |
| `StandardScaler` | Standardizes features (z-score normalization) |
| `model.fit(X, y)` | Train the model |
| `model.predict(X)` | Make predictions |
| `model.coef_` | Learned weight vector (one per feature) |
| `model.intercept_` | Learned bias |
| `mean_squared_error` | Compute MSE |
| `r2_score` | Compute R² (1.0 = perfect fit) |

### What's New in Level 2

- `StandardScaler` replaces your manual `zscore_normalize` function
- `LinearRegression` handles any number of features
- `model.coef_` is now a vector (one weight per feature)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

np.random.seed(42)

## 2.1 Generate the Same Synthetic Data

We use the same multi-feature data as Level 1 so we can directly compare results.

True relationship: $y = 0.5 x_1 + 2.0 x_2 + 0.1 x_3 + 4 + \text{noise}$

In [ ]:
m = 1000
n = 3

X_train = np.column_stack([
    500 + 3000 * np.random.rand(m),
    1 + 4 * np.random.rand(m),
    100 * np.random.rand(m),
])

w_true = np.array([0.5, 2.0, 0.1])
b_true = 4.0

y_train = X_train @ w_true + b_true + np.random.randn(m) * 5

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

## 2.2 Feature Scaling with StandardScaler

In Level 1, you implemented z-score normalization manually. scikit-learn provides `StandardScaler` to do this.

**Important**: Fit the scaler on training data only, then use `transform()` on both train and test data.

In [ ]:
# TODO: Scale features using StandardScaler
# Step 1: Create a StandardScaler instance
# Step 2: Fit and transform X_train using fit_transform()
# Step 3: Print the mean and std before/after scaling

raise NotImplementedError("Implement feature scaling with StandardScaler")

## 2.3 Fit the Model

Create a `LinearRegression` model and fit it on the scaled data.

Note: scikit-learn's `LinearRegression` uses the **Normal Equation** (closed-form), not gradient descent. It doesn't strictly require feature scaling, but scaling still helps with numerical stability and interpreting coefficients.

In [ ]:
# TODO: Create and fit a LinearRegression model
# Step 1: Create a LinearRegression instance
# Step 2: Fit on X_train_scaled and y_train
# Step 3: Print the learned weights (model.coef_) and bias (model.intercept_)

raise NotImplementedError("Implement model fitting")

## 2.4 Make Predictions and Evaluate

In [ ]:
# TODO: Make predictions and compute metrics
# Step 1: Predict using model.predict(X_train_scaled)
# Step 2: Compute MSE using mean_squared_error()
# Step 3: Compute R² using r2_score()
# Step 4: Print the metrics

raise NotImplementedError("Implement predictions and evaluation")

## 2.5 Fit Without Scaling (Compare)

scikit-learn's `LinearRegression` can fit without scaling. Let's see how the coefficients compare.

In [ ]:
# TODO: Fit LinearRegression on raw (unscaled) X_train
# Step 1: Create a new LinearRegression and fit on X_train (raw)
# Step 2: Print the learned weights — they should be close to w_true!
# Step 3: Compare with the scaled version's weights

raise NotImplementedError("Implement unscaled model for comparison")

## 2.6 Visualize: Feature Weights Comparison

In [ ]:
# TODO: Create a bar chart comparing:
# 1. True weights (w_true)
# 2. sklearn weights (unscaled model — should match true weights)
# 3. Your Level 1 weights (copy from your Level 1 results)
#
# Hint: Use plt.bar() with different x positions for each group

feature_names = ["Size", "Bedrooms", "Age"]

# Your visualization code here...

## Reflection Questions

Answer these in the markdown cell below:

1. When you fit on raw data, `model.coef_` matches the true weights. When you fit on scaled data, it doesn't. Why?
2. If sklearn doesn't require scaling, why is scaling still useful in practice?
3. sklearn uses the Normal Equation (closed-form), not gradient descent. What are the trade-offs?
4. How do the coefficients from the scaled model help you understand feature importance?

*Your answers here...*

---

## Next Steps

Level 2 complete! Continue to:
- **Level 3**: `multiple_regression_real_world.ipynb` — Apply to the Diabetes dataset with production-style code